In [1]:
cd ..

/home/ayush/Documents/AI/Projects/Toxic-Comment-Analyser


/home/ayush/Documents/AI/Projects/Toxic-Comment-Analyser/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from dataclasses import dataclass

@dataclass
class ModelEvalutionConfig:
    root_dir: str
    clean_data_yt_path: str
    tokenizer_path:str
    model_path:str


In [3]:
from src.constants import *
from src.utils.common import read_yaml_file,create_dir

In [4]:
class ConfigurationManager:
    def __init__(self,config=CONFIG_FILE_PATH,
                 param=PARAM_FILE_PATH):
        self.config=read_yaml_file(config)
        self.param=read_yaml_file(param)
        create_dir([self.config.artifact_root])

    def get_model_eval(self)->ModelEvalutionConfig:
        config = self.config.model_evalution
        create_dir([config.root_dir])
        get_modelEval=ModelEvalutionConfig(
            root_dir=config.root_dir,
            clean_data_yt_path=config.clean_data_yt_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path

        )
        return get_modelEval

In [5]:
from collections import Counter
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
from src.logging import logger

/home/ayush/Documents/AI/Projects/Toxic-Comment-Analyser/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-15 16:01:31.602208: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-15 16:01:31.603858: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-15 16:01:31.611218: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-15 16:01:31.628317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuF

In [6]:
class ModelEvalution:
    def __init__(self,config=ModelEvalutionConfig):
        self.config=config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.config.model_path)
        self.pipeline =  TextClassificationPipeline(model=self.model, tokenizer=self.tokenizer)

        with open(self.config.clean_data_yt_path, "r", encoding="utf-8") as f:
                self.text = [line.strip() for line in f.readlines()]
                logger.info("yt Comment Text successfully loaded")
        
    def predict(self):
        result = [self.pipeline(c) for c in self.text]
        flat_preds = [x[0] for x in result]
        labels = [entry['label'] for entry in flat_preds]
        scores = [entry['score'] for entry in flat_preds]
        most_common_label = Counter(labels).most_common(1)[0][0]
        most_common_label_count = Counter(labels).most_common(1)[0][1]
        percentage = (most_common_label_count/len(labels))*100
        return f"Your comments are {percentage:.1f}% {most_common_label} with an average model confidence of {np.mean(scores):.2f}."
            

In [7]:
config = ConfigurationManager()
get_modelEval= config.get_model_eval()
modeleval = ModelEvalution(config=get_modelEval)
modeleval.predict()

[2025-06-15 16:01:38,329 : INFO : common : Created directory at: artifacts]
[2025-06-15 16:01:38,335 : INFO : common : Created directory at: artifacts/model_evalution]


Device set to use cpu


[2025-06-15 16:01:39,227 : INFO : 2242629154 : yt Comment Text successfully loaded]


'Your comments are 94.2% non-toxic with an average model confidence of 0.97.'